In [0]:
import pandas as pd
import json
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql import functions as sf
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.master("local").appName("NikeTest").getOrCreate()

In [0]:
calendar_raw = spark.read.csv("/FileStore/tables/calendar.csv",header=True)
product_raw = spark.read.csv('/FileStore/tables/product.csv',header=True)
store_raw = spark.read.csv('/FileStore/tables/store.csv',header=True)
sales_raw = spark.read.csv('/FileStore/tables/sales.csv',header=True)

In [0]:
#check for duplicates
print(f'the original number of rows in calendar data {calendar_raw.count()}')
print(f'the number of duplicates in calendar = {calendar_raw.count() - calendar_raw.dropDuplicates().count()}')

print(f'the original number of rows in sales data {sales_raw.count()}')
print(f'the number of duplicates in sales = {sales_raw.count() - sales_raw.dropDuplicates().count()}')

print(f'the original number of rows in product data {product_raw.count()}')
print(f'the number of duplicates in product = {product_raw.count() - product_raw.dropDuplicates().count()}')

print(f'the original number of rows in store data {store_raw.count()}')
print(f'the number of duplicates in store = {store_raw.count() - store_raw.dropDuplicates().count()}')

the original number of rows in calendar data 365
the number of duplicates in calendar = 0
the original number of rows in sales data 70
the number of duplicates in sales = 0
the original number of rows in product data 20
the number of duplicates in product = 0
the original number of rows in store data 20
the number of duplicates in store = 0

In [0]:
#combine all data
combined_data = sales_raw.join(calendar_raw, sales_raw.dateId == calendar_raw.datekey, how = 'inner')\
                    .join(product_raw, sales_raw.productId == product_raw.productid, how = 'inner')\
                    .join(store_raw, sales_raw.storeId == store_raw.storeid, how = 'inner')
combined_data.show(10)

+------+--------+----------+-------+------+--------------+-------+---------------+----------------+------------------+--------------+--------+------+-----------+-------+-------+------------+
saleId|netSales|salesUnits|storeId|dateId| productId|datekey|datecalendarday|datecalendaryear|weeknumberofseason| productid|division|gender| category|storeid|channel| country|
+------+--------+----------+-------+------+--------------+-------+---------------+----------------+------------------+--------------+--------+------+-----------+-------+-------+------------+
 1124| 300.24| 5| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 1123| 700.24| 5| 419| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 419|Digital|LUXEMBOURG |
 1122| 500.24| 7| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 1121| 400.24| 5| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 1120| 300.24| 5| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 1119| 300.24| 5| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 1118| 300.24| 8| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 1117| 300.24| 2| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 1116| 300.24| 5| 409| 4965|00565177969059| 4965| 1| 2018| 2|00565177969059| APPAREL| KIDS|COLLECTIONS| 409|Digital| INDIA|
 136| 900.24| 5| 419| 4965|00565177969035| 4965| 1| 2018| 2|00565177969035|FOOTWEAR| MENS|COLLECTIONS| 419|Digital|LUXEMBOURG |
+------+--------+----------+-------+------+--------------+-------+---------------+----------------+------------------+--------------+--------+------+-----------+-------+-------+------------+
only showing top 10 rows

In [0]:
#create unique column
combined_data = combined_data.withColumn('uniqueKey',sf.concat(sf.lit('RY'),combined_data.datecalendaryear.substr(3,2)\
                         ,sf.lit('_'), combined_data.channel, sf.lit('_'), combined_data.division, sf.lit('_')\
                         ,combined_data.gender, sf.lit('_'), combined_data.category))

In [0]:
combined_data = combined_data.withColumn('netSales', combined_data['netSales'].cast(IntegerType()))
combined_data = combined_data.withColumn('salesUnits', sf.col('salesUnits').cast('integer'))

In [0]:
combined_data.describe()

Out[48]: DataFrame[summary: string, saleId: string, netSales: string, salesUnits: string, storeId: string, dateId: string, productId: string, datekey: string, datecalendarday: string, datecalendaryear: string, weeknumberofseason: string, productid: string, division: string, gender: string, category: string, storeid: string, channel: string, country: string, uniqueKey: string]

In [0]:
#Aggregate at unique key, divison, gender , category, channel and weeknumber of season - sum netSales, salesUnits
calc_data = combined_data.groupBy('uniqueKey','division','gender','category'\
                                  ,'channel','weeknumberofseason')\
                                    .sum('netSales','salesUnits')

In [0]:
calc_data.columns

Out[53]: ['uniqueKey',
 'division',
 'gender',
 'category',
 'channel',
 'weeknumberofseason',
 'sum(netSales)',
 'sum(salesUnits)']

In [0]:
calc_data = calc_data.withColumn('year', calc_data.uniqueKey.substr(1,4))

In [0]:
#convert to Pandas dataframe
calc_data = calc_data.toPandas()

In [0]:
calc_data = calc_data.rename(columns={'sum(netSales)':'netSales','sum(salesUnits)':'salesUnits'})

In [0]:
#Create List and Dictionary for the result json file
insert = {}
datarow1 = {}
datarow2 = {}

for i in range(calc_data.shape[0]):
    insert['uniqueKey'] = calc_data.loc[i,'uniqueKey']
    insert['division'] = calc_data.loc[i,'division']
    insert['gender'] = calc_data.loc[i,'gender']
    insert['category'] = calc_data.loc[i,'category']
    insert['channel'] = calc_data.loc[i,'channel']
    insert['year'] = calc_data.loc[i,'year']
    insert['dataRow'] = {}
    datarow1['rowId'] = 'netSales'
    datarow1['dataRow'] = {}
    datarow2['rowId'] = 'salesUnits'
    datarow2['dataRow'] = {}
    for week in range(1,53):
        if str(week) == calc_data.loc[i,'weeknumberofseason']:
            datarow1['dataRow']['W'+ str(calc_data.loc[i,'weeknumberofseason'])] = str(calc_data.loc[i,'netSales'])
            datarow2['dataRow']['W'+ str(calc_data.loc[i,'weeknumberofseason'])] = str(calc_data.loc[i,'salesUnits'])
        else:
            datarow1['dataRow']['W'+ str(week)] = str(0)
            datarow2['dataRow']['W'+ str(week)] = str(0)
    insert['dataRow'] = [datarow1,datarow2]
    print(insert,"\n\n\n")
    jsonFile = 'consumption_' + insert['uniqueKey'] + '.json'
    with open(jsonFile,'w') as f:
        json.dump(insert,f) 

{'uniqueKey': 'RY18_Digital_FOOTWEAR_MENS_COLLECTIONS', 'division': 'FOOTWEAR', 'gender': 'MENS', 'category': 'COLLECTIONS', 'channel': 'Digital', 'year': 'RY18', 'dataRow': [{'rowId': 'netSales', 'dataRow': {'W1': '0', 'W2': '3765', 'W3': '0', 'W4': '0', 'W5': '0', 'W6': '0', 'W7': '0', 'W8': '0', 'W9': '0', 'W10': '0', 'W11': '0', 'W12': '0', 'W13': '0', 'W14': '0', 'W15': '0', 'W16': '0', 'W17': '0', 'W18': '0', 'W19': '0', 'W20': '0', 'W21': '0', 'W22': '0', 'W23': '0', 'W24': '0', 'W25': '0', 'W26': '0', 'W27': '0', 'W28': '0', 'W29': '0', 'W30': '0', 'W31': '0', 'W32': '0', 'W33': '0', 'W34': '0', 'W35': '0', 'W36': '0', 'W37': '0', 'W38': '0', 'W39': '0', 'W40': '0', 'W41': '0', 'W42': '0', 'W43': '0', 'W44': '0', 'W45': '0', 'W46': '0', 'W47': '0', 'W48': '0', 'W49': '0', 'W50': '0', 'W51': '0', 'W52': '0'}}, {'rowId': 'salesUnits', 'dataRow': {'W1': '0', 'W2': '55', 'W3': '0', 'W4': '0', 'W5': '0', 'W6': '0', 'W7': '0', 'W8': '0', 'W9': '0', 'W10': '0', 'W11': '0', 'W12': '0', 'W13': '0', 'W14': '0', 'W15': '0', 'W16': '0', 'W17': '0', 'W18': '0', 'W19': '0', 'W20': '0', 'W21': '0', 'W22': '0', 'W23': '0', 'W24': '0', 'W25': '0', 'W26': '0', 'W27': '0', 'W28': '0', 'W29': '0', 'W30': '0', 'W31': '0', 'W32': '0', 'W33': '0', 'W34': '0', 'W35': '0', 'W36': '0', 'W37': '0', 'W38': '0', 'W39': '0', 'W40': '0', 'W41': '0', 'W42': '0', 'W43': '0', 'W44': '0', 'W45': '0', 'W46': '0', 'W47': '0', 'W48': '0', 'W49': '0', 'W50': '0', 'W51': '0', 'W52': '0'}}]} 



{'uniqueKey': 'RY18_Digital_APPAREL_KIDS_CRICKET', 'division': 'APPAREL', 'gender': 'KIDS', 'category': 'CRICKET', 'channel': 'Digital', 'year': 'RY18', 'dataRow': [{'rowId': 'netSales', 'dataRow': {'W1': '0', 'W2': '12000', 'W3': '0', 'W4': '0', 'W5': '0', 'W6': '0', 'W7': '0', 'W8': '0', 'W9': '0', 'W10': '0', 'W11': '0', 'W12': '0', 'W13': '0', 'W14': '0', 'W15': '0', 'W16': '0', 'W17': '0', 'W18': '0', 'W19': '0', 'W20': '0', 'W21': '0', 'W22': '0', 'W23': '0', 'W24': '0', 'W25': '0', 'W26': '0', 'W27': '0', 'W28': '0', 'W29': '0', 'W30': '0', 'W31': '0', 'W32': '0', 'W33': '0', 'W34': '0', 'W35': '0', 'W36': '0', 'W37': '0', 'W38': '0', 'W39': '0', 'W40': '0', 'W41': '0', 'W42': '0', 'W43': '0', 'W44': '0', 'W45': '0', 'W46': '0', 'W47': '0', 'W48': '0', 'W49': '0', 'W50': '0', 'W51': '0', 'W52': '0'}}, {'rowId': 'salesUnits', 'dataRow': {'W1': '0', 'W2': '199', 'W3': '0', 'W4': '0', 'W5': '0', 'W6': '0', 'W7': '0', 'W8': '0', 'W9': '0', 'W10': '0', 'W11': '0', 'W12': '0', 'W13': '0', 'W14': '0', 'W15': '0', 'W16': '0', 'W17': '0', 'W18': '0', 'W19': '0', 'W20': '0', 'W21': '0', 'W22': '0', 'W23': '0', 'W24': '0', 'W25': '0', 'W26': '0', 'W27': '0', 'W28': '0', 'W29': '0', 'W30': '0', 'W31': '0', 'W32': '0', 'W33': '0', 'W34': '0', 'W35': '0', 'W36': '0', 'W37': '0', 'W38': '0', 'W39': '0', 'W40': '0', 'W41': '0', 'W42': '0', 'W43': '0', 'W44': '0', 'W45': '0', 'W46': '0', 'W47': '0', 'W48': '0', 'W49': '0', 'W50': '0', 'W51': '0', 'W52': '0'}}]} 



{'uniqueKey': 'RY18_Digital_APPAREL_KIDS_COLLECTIONS', 'division': 'APPAREL', 'gender': 'KIDS', 'category': 'COLLECTIONS', 'channel': 'Digital', 'year': 'RY18', 'dataRow': [{'rowId': 'netSales', 'dataRow': {'W1': '0', 'W2': '8187', 'W3': '0', 'W4': '0', 'W5': '0', 'W6': '0', 'W7': '0', 'W8': '0', 'W9': '0', 'W10': '0', 'W11': '0', 'W12': '0', 'W13': '0', 'W14': '0', 'W15': '0', 'W16': '0', 'W17': '0', 'W18': '0', 'W19': '0', 'W20': '0', 'W21': '0', 'W22': '0', 'W23': '0', 'W24': '0', 'W25': '0', 'W26': '0', 'W27': '0', 'W28': '0', 'W29': '0', 'W30': '0', 'W31': '0', 'W32': '0', 'W33': '0', 'W34': '0', 'W35': '0', 'W36': '0', 'W37': '0', 'W38': '0', 'W39': '0', 'W40': '0', 'W41': '0', 'W42': '0', 'W43': '0', 'W44': '0', 'W45': '0', 'W46': '0', 'W47': '0', 'W48': '0', 'W49': '0', 'W50': '0', 'W51': '0', 'W52': '0'}}, {'rowId': 'salesUnits', 'dataRow': {'W1': '0', 'W2': '111', 'W3': '0', 'W4': '0', 'W5': '0', 'W6': '0', 'W7': '0', 'W8': '0', 'W9': '0', 'W10': '0', 'W11': '0', 'W12': '0', 'W13': '0', 'W14': '0', 'W15': '